### Object tracking with opencv

In [2]:
import cv2

#### default

In [3]:
## DEFAULT WITH LOTS OF NOISE
cap = cv2.VideoCapture('../data/aerial_tennis_video.mp4')
back = cv2.createBackgroundSubtractorMOG2(varThreshold=50, history=20)

while True:
    # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)               
        if ret:
        # Apply background subtraction
            fg_mask = back.apply(frame)                                                     # BACKGROUND MASK
            cv2.imshow('Frame_bg', fg_mask)    
                                
            # Find contours
            contours, hierarchy = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # print(contours)
            frame_ct = cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)
            # Display the resulting frame
            cv2.imshow('Frame_final', frame_ct)

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


[-1, 113, -1, -1]

#### with adjusted settings

In [16]:
## MORE SETTINGS
cap = cv2.VideoCapture('../data/aerial_tennis_video.mp4')
back = cv2.createBackgroundSubtractorMOG2(varThreshold=700, history=20)

while True:
    # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Stream ended. Exiting...")
            break        
          
        if ret:
            cv2.imshow('start', frame)    

            fg_mask = back.apply(frame)                                                     # BACKGROUND MASK
            cv2.imshow('Frame_bg', fg_mask)    

            retval, mask_thresh = cv2.threshold(fg_mask, 180, 255, cv2.THRESH_BINARY)       # TRESHOLD (less noise and shadows)
            cv2.imshow('frame_thresh', mask_thresh)            

            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
            mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)
            cv2.imshow('frame_erode', mask_eroded)                                          # EROSION
                                
            contours, hierarchy = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # min_contour_area = 1                                                                                                            
            # large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area] # AREA TRESHOLD 
            # contours = [cnt for cn in contours]
            # frame_ct = cv2.drawContours(frame, contours, -1, (0, 0, 255), 2)
            # cv2.imshow('frame_all_contours', frame_ct)                                         

            frame_out = frame.copy()
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                wi = w * 5                                                                 # INCREASE BBOX SIZE FOR VISUAL
                hi = h * 5
                frame_out = cv2.rectangle(frame, (x, y), (x+wi, y+hi), (0, 0, 255), 3)
            
            # Display the resulting frame
            cv2.imshow('Frame_final', frame_out)            

            key = cv2.waitKey(1)
            if key == ord('q'):                                         # exit
                break        
            elif key == ord('x'):                                       # pauze screen
                cv2.waitKey(0)                 

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


[-1, -1, -1, -1]

In [ ]:
#https://learnopencv.com/moving-object-detection-with-opencv/
#### steps to take for tracking:

# background substraction
# global tresholding
# erotion and dilation
# contour detection and filtering
# bounding box